# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 85 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.09782


extracting tarball to tmp_2505.09782...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.09879
extracting tarball to tmp_2505.09879... done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2505.09782/virgoha_astroph.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'NGC5364_tab1' from 'tmp_2505.09782/NGC5364_tab1.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Retrieving document from  https://arxiv.org/e-print/2505.10395


extracting tarball to tmp_2505.10395... done.
Retrieving document from  https://arxiv.org/e-print/2505.10416


extracting tarball to tmp_2505.10416... done.


Found 77 bibliographic references in tmp_2505.10416/AandA.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10416-b31b1b.svg)](https://arxiv.org/abs/2505.10416) | **Euclid: Photometric redshift calibration with the clustering redshifts technique**  |
|| W. d. Doumerg, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-05-16*|
|*Comments*| *32 pages, 26 figures, 2 tables*|
|**Abstract**|            Aims: The precision of cosmological constraints from imaging surveys hinges on accurately estimating the redshift distribution $ n(z) $ of tomographic bins, especially their mean redshifts. We assess the effectiveness of the clustering redshifts technique in constraining Euclid tomographic redshift bins to meet the target uncertainty of $ \sigma ( \langle z \rangle ) < 0.002 (1 + z) $. In this work, these mean redshifts are inferred from the small-scale angular clustering of Euclid galaxies, which are distributed into bins with spectroscopic samples localised in narrow redshift slices. Methods: We generate spectroscopic mocks from the Flagship2 simulation for the Baryon Oscillation Spectroscopic Survey (BOSS), the Dark Energy Spectroscopic Instrument (DESI), and Euclid's Near-Infrared Spectrometer and Photometer (NISP) spectroscopic survey. We evaluate and optimise the clustering redshifts pipeline, introducing a new method for measuring photometric galaxy bias (clustering), which is the primary limitation of this technique. Results: We have successfully constrained the means and standard deviations of the redshift distributions for all of the tomographic bins (with a maximum photometric redshift of 1.6), achieving precision beyond the required thresholds. We have identified the main sources of bias, particularly the impact of the 1-halo galaxy distribution, which imposed a minimal separation scale of 1.5 Mpc for evaluating cross-correlations. These results demonstrate the potential of clustering redshifts to meet the precision requirements for Euclid, and we highlight several avenues for future improvements.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.09782-b31b1b.svg)](https://arxiv.org/abs/2505.09782) | **Virgo Filaments V: Disrupting the Baryon Cycle in the NGC 5364 Galaxy Group**  |
|| R. A. Finn, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-05-16*|
|*Comments*| *21 pages, 11 figures, accepted for publication in the Astrophysical Journal*|
|**Abstract**|            The Virgo Filament Survey (VFS) is a comprehensive study of galaxies that reside in the extended filamentary structures surrounding the Virgo Cluster, out to 12 virial radii. The primary goal is to characterize all of the dominant baryonic components within galaxies and to understand whether and how they are affected by the filament environment. A key constituent of VFS is a narrowband H$\alpha$ imaging survey of over 600 galaxies, VFS-H$\alpha$. The H$\alpha$ images reveal detailed, resolved maps of the ionized gas and massive star-formation. This imaging is particularly powerful as a probe of environmentally-induced quenching because different physical processes affect the spatial distribution of star formation in different ways. In this paper, we present the first results from the VFS-H$\alpha$ for the NGC~5364 group, a low-mass ($\log_{10}(M_{dyn}/M_\odot) < 13)$ system located at the western edge of the Virgo~III filament. We combine H$\alpha$ imaging with resolved H~I observations from MeerKAT for eight group members. These galaxies exhibit peculiar morphologies, including strong distortions in the stars and the gas, truncated H~I and H$\alpha$ disks, H~I tails, extraplanar H$\alpha$ emission, and off-center H$\alpha$ emission. These signatures are suggestive of environmental processing such as tidal interactions, ram pressure stripping, and starvation. We quantify the role of ram pressure stripping expected in this group, and find that it can explain the cases of H~I tails and truncated H-alpha for all but one of the disk-dominated galaxies. Our observations indicate that multiple physical mechanisms are disrupting the baryon cycle in these group galaxies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.09879-b31b1b.svg)](https://arxiv.org/abs/2505.09879) | **New cosmological constraints on the evolution of dark matter energy density**  |
|| Y. Yang, X. Dai, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-16*|
|*Comments*| *11 pages, 1 figures, 2 tables. Accepted by PRD (submitted on 28. Feb. 2025). Comments are still welcome. In particular, if you discover that your research is relevant to our work and your findings are not currently listed in Table II of our manuscript, we would greatly appreciate it if you could contact us. We will gladly include your results in the arXiv version of Table II*|
|**Abstract**|            We constrain the evolution of dark matter energy density over time, specifically focusing on deviation from the standard model represented by the equation $\rho_{m}\propto(1+z)^{3-\varepsilon}$, where $\varepsilon$ is a constant parameter. Utilizing a diverse array of observational datasets, including baryon acoustic oscillations (BAO) data from the first release of the Dark Energy Spectroscopic Instrument (DESI), distance priors derived from cosmic microwave background (CMB) observations by the Planck satellite, Hubble rate data obtained through the cosmic chronometers (CC) method, type Ia supernova (SNIa) data from the Panthon sample, and the data from the redshift space distortion (RSD) measurements ($f\sigma_8$), we derive stringent constraints on the deviation parameter. We find that for the model under consideration, the deviation parameter is constrained to be $\varepsilon = -0.0073^{+0.0029}_{-0.0033}$, indicating a deviation of approximately $2.4\sigma$ from the scenario where dark matter and vacuum dark energy do not interact. When compared with previous studies and alternative analyses, our findings provide corroborative evidence for an interaction between dark matter and vacuum dark energy, particularly in light of the release of BAO data from DESI.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10395-b31b1b.svg)](https://arxiv.org/abs/2505.10395) | **S241125n: Binary Black Hole Merger Produces Short GRB in AGN Disk**  |
|| S.-R. Zhang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-16*|
|*Comments*| *14 pages, 8 figures, submitted to ApJL. Comments welcome!*|
|**Abstract**|            Recently, the gravitational wave (GW) event for black hole (BH)-BH mergers, S241125n, has been reported to be associated with a short gamma-ray burst (GRB) and X-ray afterglow emission. Such an association could potentially unveil the environments of mergers and provide attractive targets for multi-messenger observations. We summarize these observations and model it as a binary BH merger occurring within an active galactic nucleus (AGN), where the merger remnant accretes disk material at hyper-Eddington rates. The resulting jet could lead to the GRB associated with the GW event. The GRB, detected by Swift-BAT, exhibits a Comptonized spectrum with an unusually soft photon index, which is consistent with emission from the jet interacting the emission associated with the dense AGN disk environment. Its X-ray afterglow shows an unusually hard spectrum, indicating possible strong absorption by a high column density typical of AGN disks. We highlight the importance of identifying the host galaxy and conducting more sensitive infrared observations to test the model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.10416/./Figures/nz_true_shape_bin1_npl.png', 'tmp_2505.10416/./Figures/Wdm_theory_npl.png', 'tmp_2505.10416/./Figures/Bias_dz10_npl.png']
copying  tmp_2505.10416/./Figures/nz_true_shape_bin1_npl.png to _build/html/
copying  tmp_2505.10416/./Figures/Wdm_theory_npl.png to _build/html/
copying  tmp_2505.10416/./Figures/Bias_dz10_npl.png to _build/html/
exported in  _build/html/2505.10416.md
    + _build/html/tmp_2505.10416/./Figures/nz_true_shape_bin1_npl.png
    + _build/html/tmp_2505.10416/./Figures/Wdm_theory_npl.png
    + _build/html/tmp_2505.10416/./Figures/Bias_dz10_npl.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dz}{\mathrm{d}z}$
$\newcommand{\w}{\omega}$
$\newcommand{\william}[1]{{\textcolor{purple}{[WdA: #1]}}}$
$\newcommand{\corr}[1]{{\textcolor{red}{ #1}}}$
$\newcommand{\smallO}[1]{\ensuremath{\mathop \mathopen{\scriptstyle\mathcal{O}}\mathopen \left(#1\right)}}$
$\newcommand{\bigO}[1]{\ensuremath{\mathop \mathopen{ \Large\mathcal{O}}\mathopen \left(#1\right)}}$
$\newcommand{\orcid}[1]$
$\newcommand{\d}{\delta}$
$\newcommand{\l}{\langle}$
$\newcommand{\r}{\rangle}$
$\newcommand{\np}{n_{\rm p}}$
$\newcommand{\wsp}{w_{\rm sp}}$
$\newcommand{\bp}{b_{\rm p}}$
$\newcommand{\bs}{b_{\rm s}}$
$\newcommand{\thetamin}{\theta_{\rm min}}$
$\newcommand{\thetamax}{\theta_{\rm max}}$
$\newcommand{\drp}{{\rm d}r_{\rm p}}$
$\newcommand{\rp}{r_{\rm p}}$
$\newcommand{\rpmin}{r_{\rm p}^{\rm min}}$
$\newcommand{\rpmax}{r_{\rm p}^{\rm max}}$</div>



<div id="title">

# $\Euclid$\/: Photometric  redshift calibration with the clustering redshifts technique $\thanks{This paper is published on behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.10416-b31b1b.svg)](https://arxiv.org/abs/2505.10416)<mark>Appeared on: 2025-05-16</mark> -  _32 pages, 26 figures, 2 tables_

</div>
<div id="authors">

W. d. Doumerg, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Aims: The precision of cosmological constraints from imaging surveys hinges on accurately estimating the redshift distribution $n(z)$ of tomographic bins, especially their mean redshifts. We assess the effectiveness of the clustering redshifts technique in constraining $\Euclid$ tomographic redshift bins to meet the target uncertainty of $ \sigma(\langle z \rangle)< 0.002 (1 + z)$ . In this work, these mean redshifts are inferred from the small-scale angular clustering of $\Euclid$ galaxies, which are distributed into bins with spectroscopic samples localised in narrow redshift slices.Methods: We generate spectroscopic mocks from the Flagship2 simulation for the Baryon Oscillation Spectroscopic Survey (BOSS), the Dark Energy Spectroscopic Instrument (DESI), and _Euclid_ ’s Near-Infrared Spectrometer and Photometer (NISP) spectroscopic survey. We evaluate and optimise the clustering redshifts pipeline, introducing a new method for measuring photometric galaxy bias (clustering), which is the primary limitation of this technique.Results: We have successfully constrained the means and standard deviations of the redshift distributions for all of the tomographic bins (with a maximum photometric redshift of 1.6), achieving precision beyond the required thresholds. We have identified the main sources of bias, particularly the impact of the 1-halo galaxy distribution, which imposed a minimal separation scale of 1.5 Mpc for evaluating cross-correlations. These results demonstrate the potential of clustering redshifts to meet the precision requirements for $\Euclid$ , and we highlight several avenues for future improvements.

</div>

<div id="div_fig1">

<img src="tmp_2505.10416/./Figures/nz_true_shape_bin1_npl.png" alt="Fig2" width="100%"/>

**Figure 2. -**  A galaxy distribution observed through different $\Delta z$ averaging as Eq. \eqref{eq:nz_avg}. The redshift convolution is modifying the shape of the distribution for larger slicing, in comparison to the real one estimated with a very thin slicing (grey). For that case, all slicing $\Delta z $\l$eq 0.06$ produce a good estimate. (*fig:nz_binning*)

</div>
<div id="div_fig2">

<img src="tmp_2505.10416/./Figures/Wdm_theory_npl.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Top_: Redshift distribution of sample $b$ in blue, and four out of the ten samples $a_i$ in red. _Bottom_: Deviation of approximated $w_{a_ib}$ w.r.t. the exact one, for the 10 cross-correlations. (*fig:Toy_model_wab*)

</div>
<div id="div_fig3">

<img src="tmp_2505.10416/./Figures/Bias_dz10_npl.png" alt="Fig7" width="100%"/>

**Figure 7. -**  True $w_{$\r$m pp}$ for $\Delta z=0.05$ in grey and measurement with corrections factors including (M3) in green, blue, and orange dashed lines for photo-$z$ binning $\Delta z=0.1$. The best fits from degree-3 polynomials are reported with solid lines. The $w_{$\r$m pp}$ used for  (M3) is the best-fit model of the full correction (solid red line).   (*fig:M3bias*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.10416"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

490  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
